In [0]:
!pip install geopandas
!pip install descartes
!pip install mplleaflet
!pip install seaborn

     |████████████████████████████████| 901kB 5.0MB/s 
     |████████████████████████████████| 17.9MB 38.1MB/s 
     |████████████████████████████████| 11.2MB 33.2MB/s 
  Created wheel for munch: filename=munch-2.3.2-py2.py3-none-any.whl size=6613 sha256=0a6fe59819d86bf244b8443ffd3e91212bf8fc2e5a98f7b68fa33aeae810a27c
  Stored in directory: /root/.cache/pip/wheels/db/bf/bc/06a3e1bfe0ab27d2e720ceb3cff3159398d92644c0cec2c125
Successfully built munch
  Created wheel for mplleaflet: filename=mplleaflet-0.0.5-cp36-none-any.whl size=28555 sha256=356687389b93524574e9ad6e0bef1a70f7a870be68cb5890178deaafa2cc4efe
  Stored in directory: /root/.cache/pip/wheels/bf/fd/0b/0513de62c339c4a56e2234aa42a5d06e6660a4446fdae7b796
Successfully built mplleaflet


In [0]:
import numpy as np
import pandas as pd
import geopandas as gpd
from functools import reduce 

import time

import matplotlib as mp
import matplotlib.pyplot as plt
import mplleaflet
import descartes
import seaborn as sns
pd.options.mode.chained_assignment = None

import warnings
warnings.filterwarnings('ignore')

sns.set(style="whitegrid")
sns.set(font_scale=2)

In [0]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [0]:
from google.cloud import bigquery
client = bigquery.Client('thesisprojects2019')

In [0]:
#import london statistic table from bigquery
query = """ SELECT * FROM `thesisprojects2019.audeMS.step3_statistic_county_homelocation_spectatorswithhome` ORDER BY eventid, county_id """
data = client.query(query).to_dataframe()

In [0]:
data.head(2)

,venueid,eventid,event,county_id,county_name,count_spect_county,perc_spect_county
0,1,1_2017-11-04,Barbarians New Zealand,E06000002,Middlesbrough,2,0.268
1,1,1_2017-11-04,Barbarians New Zealand,E06000015,Derby,1,0.134


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#import the london shapefiles from GoogleDrive
pathdrive = 'drive/My Drive/Colab Notebooks/Shapefiles/County/Counties_and_Unitary_Authorities_December_2015_Ultra_Generalised_Clipped_Boundaries_in_England_and_Wales.shp'
geom_data = gpd.read_file(pathdrive)

In [0]:
geom_data.head(2)

,ctyua15cd,ctyua15nm,ctyua15nmw,objectid,st_lengths,st_areasha,geometry
0,E06000001,Hartlepool,None,1,50778.093996,9.633958e+07,"POLYGON ((448906.5049000001 536702.9926999994,..."
1,E06000002,Middlesbrough,None,2,35500.301293,5.523094e+07,"POLYGON ((451894.2999999998 521145.3000000007,..."


In [0]:
len(geom_data)

174

In [0]:
# CRS for London Ward
crs = {'init': 'epsg:4326'}
county = geom_data.to_crs({'init':'epsg:27700'})

# rename the columns
county = county.rename(columns={'ctyua15nm':'county_name','ctyua15cd':'county_id'})
# Select only the columns of interest
county = county[['county_id','county_name','geometry']]

In [0]:
county.head(2)

,county_id,county_name,geometry
0,E06000001,Hartlepool,"POLYGON ((448906.5049000001 536702.9926999994,..."
1,E06000002,Middlesbrough,"POLYGON ((451894.2999999998 521145.3000000007,..."


In [0]:
# Join Districts SHP with data_homelocation_count
jointable = pd.merge(data,county,how='right', on=['county_id'])
jointable = jointable[['venueid','eventid','event','county_id','county_name_x','perc_spect_county','geometry']]
jointable = jointable.sort_values('county_id').reset_index(drop=True)
jointable['perc_spect_county'] = round(jointable['perc_spect_county'],3)

In [0]:
jointable.head(2)

,venueid,eventid,event,county_id,county_name_x,perc_spect_county,geometry
0,1.0,1_2017-11-18,England Australia,E06000001,Hartlepool,0.081,"POLYGON ((448906.5049000001 536702.9926999994,..."
1,4.0,4_2017-09-24,NFL Game 1,E06000001,Hartlepool,0.069,"POLYGON ((448906.5049000001 536702.9926999994,..."


In [0]:
# Get the list of all the events
event_list = data.eventid.unique()

In [0]:
#Download images into google drive
from google.colab import files

In [0]:
#def GetDataFrame(datasource):
  # Selection event
for e in event_list:
    # Mergedata 
    dataevent = data[data['eventid']==e]
    datajoin = pd.merge(dataevent,county,how='right',on=['county_id'])
    datajoin = datajoin[['venueid','eventid','event','county_id','county_name_x','perc_spect_county','geometry']]
    
    # select the event name
    eventname = datajoin['event'][0]
    
    # change to gdp
    gpd_df = gpd.GeoDataFrame(datajoin, geometry='geometry', crs={'init': 'epsg:27700'})
      
    # If NaN -> change it to 0!   
    values = {'venueid': 0, 'eventid':0,'perc_spect_county':0}  
    gpd_df = gpd_df.fillna(value=values)
      
    # Selection the variable to plot
    var_map = gpd_df['perc_spect_county']
      
    # Set the range for the choropleth
    vmin = gpd_df['perc_spect_county'].min()
    vmax = gpd_df['perc_spect_county'].max()
      
    # create figure and axes for Matplotlib
    fig, ax = plt.subplots(1, figsize=(8, 10), dpi=160)
   # fig, ax = plt.subplots(1, figsize=(8, 10))
 
    # create map
    gpd_df.plot(column=var_map, cmap='YlGn', linewidth=0.6, ax=ax, edgecolor='0.8')
    
 #   plt.suptitle('Event Catchment Area - Spectators Homelocation', fontsize=12, horizontalalignment = 'center')
    plt.title(str(eventname), fontsize=12, horizontalalignment='center')
    plt.xlabel('Longitude',fontsize=12, horizontalalignment='center')
    plt.ylabel('Latitude',fontsize=12, horizontalalignment='center')
    plt.xticks(horizontalalignment='center', fontsize=9)
    plt.yticks(horizontalalignment='right', fontsize=9)
    
    ax.tick_params(axis='both', which='major', pad=6)
    
    sm = plt.cm.ScalarMappable(cmap='YlGn', norm=plt.Normalize(vmin=vmin, vmax=vmax))
    sm._A = []

    cbar = fig.colorbar(sm, orientation='horizontal',fraction=0.04, pad=0.09)
    cbar.set_label('Percentage Event Spectators ', size=11)
    cbar.ax.xaxis.set_label_position('top')
    cbar.ax.tick_params(labelsize=10)
     
    fig.tight_layout()
    
    plt.savefig('drive/My Drive/Dan - Meetings/Thesis Maps & Graphs/County_Home/Home_Spectators/county_map_'+str(eventname)+'.png')
    
    plt.close('all')
  #  plt.show()

In [0]:
import os

folderpath = '/content/drive/My Drive/Dan - Meetings/Thesis Maps & Graphs/County_Home/Home_Spectators'

for filename in os.listdir(folderpath):
 # print(filename)
  files.download(folderpath+'/'+str(filename))